In [4]:
#import the needed modules
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp


In [9]:
#read the data
df = pd.read_csv("bank-additional-full.csv" , sep=";")

In [10]:
#check sample of the dat
df.head(7)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [11]:
#check the shape of the data
df.shape

(41188, 21)

In [13]:
#check all columns in our data
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [14]:
#check for null values
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [15]:
#check the data types
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [ ]:
#since we will be using boosting algorithms we will convert all categoricals ie objects to numbers
# in the above columns

In [16]:
#we will bepredicting column y.....lets check what it contains
df["y"].unique()

array(['no', 'yes'], dtype=object)

In [18]:
#lets do quick profiling of our data for review
pp.ProfileReport(df)

/opt/anaconda/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,21
Number of observations,41188
Total Missing (%),0.0%
Total size in memory,6.6 MiB
Average record size in memory,168.0 B
Numeric,8
Categorical,11
Boolean,0
Date,0
Text (Unique),0
Rejected,2


# lets encode our categorical data
+using onehot encoding and get dummies

In [23]:
#check for the data where dtype is object
df.columns[df.dtypes == "object"]

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'poutcome', 'y'],
      dtype='object')

In [24]:
#get dummies foe all columns with dtypes as objects
pd.get_dummies(df , df.columns[df.dtypes == "object"])

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,1,0,0,0,0,0,1,0,0,1
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,1,0,0,0,0,0,1,0,1,0
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,1,0,0,0,0,0,1,0,1,0
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,1,0,0,0,0,0,1,0,0,1


In [25]:
#print all unique values in each column with objecta as dtype
for x in df.columns[df.dtypes == "object"]:
    print(x , " -->" , df[x].unique())

job  --> ['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
marital  --> ['married' 'single' 'divorced' 'unknown']
education  --> ['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
default  --> ['no' 'unknown' 'yes']
housing  --> ['no' 'yes' 'unknown']
loan  --> ['no' 'yes' 'unknown']
contact  --> ['telephone' 'cellular']
month  --> ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
day_of_week  --> ['mon' 'tue' 'wed' 'thu' 'fri']
poutcome  --> ['nonexistent' 'failure' 'success']
y  --> ['no' 'yes']


In [26]:
df2 = df

In [29]:
#convert this values of categorical data to numerals
for x in  df.columns[df.dtypes == "object"]:
    #change first to categorical data type
    df2[x] = df[x].astype('category')
    df2[x] = df2[x].cat.codes

In [30]:
df2.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,3,1,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,3,0,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,1,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,3,0,0,2,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [31]:
#check for a particular data
df2.day_of_week.unique()
#we see that data has been replaced from 0 to ....-->

array([1, 3, 4, 2, 0])

In [32]:
df2.shape

(41188, 21)

In [33]:
df2.y.unique()

array([0, 1])

In [38]:
#get the features
#we will drop our target which is y
X = df2.drop('y' , axis = 1)
X.shape

(41188, 20)

In [41]:
#our target variable 
Y = df2.y
Y.shape

(41188,)

In [43]:
#lets split our data now
from sklearn.model_selection import train_test_split

In [44]:
x_train , x_test , y_train , y_test = train_test_split(X , Y , test_size = 0.25 , random_state = 0)

In [45]:
x_train.shape

(30891, 20)

### Building models
+ XGboost
+ GradiendBoostingClassifier
+ DecisionTree


In [47]:
from sklearn.ensemble import GradientBoostingClassifier
#create the object
gbc = GradientBoostingClassifier()

In [49]:
#train our data
gbc.fit(x_train , y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [51]:
#predict from the x_test data
y_predict = gbc.predict(x_test)
y_predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [52]:
#check the score of our model
from sklearn.metrics import accuracy_score

In [53]:
print("Our model Accuraccy Score is  {}".format(accuracy_score(y_test , y_predict)))

Our model Accuraccy Score is  0.9237642031659706


In [54]:
#lets check for model two using decision tree classifier
from sklearn.tree import DecisionTreeClassifier

In [55]:
#initialize the constructor
dtc = DecisionTreeClassifier()

In [56]:
#train the data
dtc.fit(x_train , y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [58]:
#check predictions
y_prediction = dtc.predict(x_test)
y_prediction

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [59]:
#check the model accuracy
print("Our model Accuraccy Score is  {}".format(accuracy_score(y_test , y_prediction)))

Our model Accuraccy Score is  0.8961833543750607


# trying XGboost

In [60]:
xgc = xgb.XGBClassifier()

In [69]:
#train our data
xgc.fit(x_train , y_train)
#shiiiiietttt

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [68]:
y_predictions = xgc.predict(x_test)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19'] ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
expected f2, f14, f15, f19, f11, f17, f13, f9, f8, f10, f1, f12, f4, f5, f3, f0, f18, f16, f7, f6 in input data
training data did not have the following fields: nr.employed, previous, cons.conf.idx, education, duration, day_of_week, poutcome, cons.price.idx, housing, contact, month, age, emp.var.rate, pdays, default, marital, euribor3m, campaign, loan, job

In [66]:
xgc.fit(x_train , x_train)

ValueError: bad input shape (30891, 20)